In [0]:
---
title: "Carbon Dioxide Level  ; time-serie study"
author: "Jonathan Bouchet"
date: "`r Sys.Date()`"
output:
 html_document:
    fig_width: 10
    fig_height: 7
    toc: yes
    number_sections : yes
    code_folding: show
---

```{r setup}
options(width=100)
knitr::opts_chunk$set(out.width='1000px',dpi=200,message=FALSE,warning=FALSE)
```

```{r}
#load packages and csv file
library(ggplot2)
library(dplyr)
library(gridExtra)
library(grid)
library(Amelia)
library(MASS)
library(caret)
library(RColorBrewer)
library(ggfortify) # plot time serie with ggplot
library(forecast)
library(imputeTS)
```

#Intro
Dataset is the record of the atmospheric carbon dioxide measured by Mauna Loa Observatory, from 1960 to now. This observatory gives a good representation of carbon dioxide level since it is located in remote locations, meaning free from local influences.

The dataset has the following columns :

* `year`, `month` and `year_decimal`
* carbon dioxide concentrations (ppm)
* carbon dioxide concentrations corrected from seasonality (ppm)
* carbon dioxide concentrations corrected from noise (ppm)
* carbon dioxide concentrations with seasonality removed (ppm)

```{r}
df<-read.csv('../input/archive.csv',sep=',')
colnames(df)<-c('year','month','decimal_date','carbon_dioxide_ppm','carbon_dioxide_ppm_season_adj','carbon_dioxide_ppm_fit','carbon_dioxide_ppm_season_adj_fit')
#missmap(df, main="Missings Map", col=c("#3B9AB2", "#EBCC2A"),y.cex = 1, x.cex = 1, legend=TRUE)
```

#Plots{.tabset .tabset-fade .tabset-pills}
##All concentrations
```{r}
ggplot() + 
  geom_line(data=df,aes(x=decimal_date,y=carbon_dioxide_ppm,color="carbon_dioxide"),alpha=.75) +
  geom_line(data=df,aes(x=decimal_date,y=carbon_dioxide_ppm_season_adj,color="carbon_dioxide_adjusted"),alpha=.75) +
  geom_line(data=df,aes(x=decimal_date,y=carbon_dioxide_ppm_fit,color="carbon_dioxide_fit"),alpha=.75) + 
  geom_line(data=df,aes(x=decimal_date,y=carbon_dioxide_ppm_season_adj_fit,color="carbon_dioxide_adjusted_fit"),alpha=.75) + 
  scale_colour_manual(name="data",values=c(carbon_dioxide="#E2D200",carbon_dioxide_adjusted="#46ACC8",carbon_dioxide_fit="#E58601",carbon_dioxide_adjusted_fit="#B40F20")) + 
  theme(legend.position="top") +
  xlab('') + ylab('concentrations [ppm]')
```

##Breakdown by corrections
```{r}
g1<-ggplot(data=df,aes(x=decimal_date,y=carbon_dioxide_ppm,color="carbon_dioxide")) + geom_line(alpha=1) + xlab('') + ylab('concentrations [ppm]') + scale_colour_manual(name="data",values=c(carbon_dioxide="#E2D200"))
g2<-ggplot(data=df,aes(x=decimal_date,y=carbon_dioxide_ppm_season_adj,color="carbon_dioxide_adjusted")) + geom_line(alpha=1) + xlab('') + ylab('concentrations [ppm]') +scale_colour_manual(name="data",values=c(carbon_dioxide_adjusted="#46ACC8"))
g3<-ggplot(data=df,aes(x=decimal_date,y=carbon_dioxide_ppm_fit,color="carbon_dioxide_fit")) + geom_line(alpha=1) + xlab('') + ylab('concentrations [ppm]') + scale_colour_manual(name="data",values=c(carbon_dioxide_fit="#E58601"))
g4<-ggplot(data=df,aes(x=decimal_date,y=carbon_dioxide_ppm_season_adj_fit,color="carbon_dioxide_adjusted_fit")) + geom_line(alpha=1) + xlab('') + ylab('concentrations [ppm]') + scale_colour_manual(name="data",values=c(carbon_dioxide_adjusted_fit="#B40F20"))

grid.draw(rbind(ggplotGrob(g1), ggplotGrob(g2),ggplotGrob(g3), ggplotGrob(g4), size = "last"))
```

##Seasonality
```{r}
mymonths <- c("January","February","March","April","May","June","July","August","September","October","November","December")
df$MonthAbb <- mymonths[ df$month ]
df$ordered_month <- factor(df$MonthAbb, levels = month.name)
ggplot(data=df, aes(x=year,y=ordered_month)) + 
  geom_tile(aes(fill = carbon_dioxide_ppm),colour = "white") +
  scale_fill_gradientn(colours=rev(brewer.pal(11,'Spectral'))) + 
  theme(axis.title.y=element_blank(),axis.title.x=element_blank(),legend.position="top")
```

##Seasonality: `box-plot` per month (group = year)
```{r}
ggplot(data=df,aes(x=ordered_month,y=carbon_dioxide_ppm)) + 
  geom_boxplot() +
  geom_jitter(shape=16, position=position_jitter(0.2)) + 
  xlab('') + ylab('concentrations [ppm]') +
  theme(axis.text.x = element_text(angle=45, hjust=1))
```

This plot is a good representation of :

* the seasonality : within a given year, the carbon concentration oscillates (peak in `may`, minima in `september/october`)
* the increase observed in 46 years

##Moving average estimation
We take the average over a year so the frequency has to be 12. The sides argument of the filter functions: 

* sides=2 uses both sides
* sides=1 uses past values only

```{r}
f12 <- rep(1/12, 12)
f6 <- rep(1/6, 6)
mav <- function(x,n=12){stats::filter(x,rep(1/n,n), sides=1)}
df$mav_1 <- stats::filter(df$carbon_dioxide_ppm, f12, sides=1)
df$mav_2 <- stats::filter(df$carbon_dioxide_ppm, f12, sides=2)
df$mav_3 <- stats::filter(df$carbon_dioxide_ppm, f6, sides=2)
#df$mav_1 <- mav(df$carbon_dioxide_ppm,1)
#df$mav_2 <- mav(df$carbon_dioxide_ppm,2)

ggplot() + 
  geom_line(data=filter(df,year>2000),aes(x=decimal_date,y=carbon_dioxide_ppm,color='carbon_dioxide')) + 
  geom_line(data=filter(df,year>2000),aes(x=decimal_date,y=carbon_dioxide_ppm_season_adj,color='carbon_dioxide_adjusted')) +
  geom_line(data=filter(df,year>2000),aes(x=decimal_date,y=mav_1,color='carbon_dioxide_ppm_moving_average_1')) +
  geom_line(data=filter(df,year>2000),aes(x=decimal_date,y=mav_2,color='carbon_dioxide_ppm_moving_average_2')) + 
  geom_line(data=filter(df,year>2000),aes(x=decimal_date,y=mav_3,color='carbon_dioxide_ppm_moving_average_2_half_year')) + 
  
  scale_colour_manual(name="data",values=c(carbon_dioxide="#E2D200",carbon_dioxide_adjusted="#46ACC8",carbon_dioxide_ppm_moving_average_1="#E58601",carbon_dioxide_ppm_moving_average_2="#B40F20",carbon_dioxide_ppm_moving_average_2_half_year="#9A8822")) + 
  theme(legend.position="top") +
  xlab('') + ylab('concentrations [ppm]')
```

Since the data has the correction per seasonality, my attempt here was just to reproduce, by looking at the moving average, if I was able to reproduce this correction. Apparently the seasonality is not by `year` (comparison of the blue and red curves)

#Time Series analysis{.tabset .tabset-fade .tabset-pills}
Since we have a large range of data, we can split them into a train/test samples in order to estimate the accuracy of each model.

```{r}
train <- df %>% dplyr::filter(year<2015) %>% dplyr::select(year,month,carbon_dioxide_ppm)
test <- df %>% dplyr::filter(year>=2015) %>% dplyr::select(year,month,carbon_dioxide_ppm)

#define the train/test time series
train_ts <- ts(train$carbon_dioxide_ppm, start=c(1958,1),end=c(2014,12),frequency=12)
test_ts <- ts(test$carbon_dioxide_ppm, start=c(2015,1),end=c(2017,12),frequency=12)

#add date-time format needed for plotting the TS with ggplot
train$dateTS<-as.Date(paste0(train$year,'-',train$month,'-01'))
test$dateTS<-as.Date(paste0(test$year,'-',test$month,'-01'))
```

There is a little work to do for using `gpplot` to draw the time-serie :

* convert the forecast into a `dataframe` (right now it's a `list`)
* make a `Date` column as (`yyy-mm-dd`) for `ggplot()` as an x-axis

```{r}
getMonth<-function(x){
	val<-strsplit(x,' ')[[1]][1]
	res<-match(tolower(val),tolower(month.abb))
	return(as.numeric(res))
}

getYear<-function(x){
	res<-strsplit(x,' ')[[1]][2]
	return(as.numeric(res))
}

convert_ts_df<-function(myts){
  #convert the forecast into a DF, convert the row index into a date column
  mydf<-data.frame(myts)
  dates<-rownames(mydf)
  rownames(mydf)<-1:nrow(mydf)  
  mydf$date<-dates
  
  #extract yea,.month as numeric and make a Date() variable
  mydf$month<-sapply(mydf$date,getMonth)
  mydf$year<-sapply(mydf$date,getYear)
  mydf$dateTS<-as.Date(paste0(mydf$year,'-',mydf$month,'-01'))
  return(mydf)
}
```

##Auto Arima 
```{r}
#arima model
m_aa = auto.arima(train_ts)
#prediction at 24 months
f_aa = forecast(m_aa, h=24)
#autoplot(f_aa)
arimaDF<-convert_ts_df(f_aa)
```

```{r}
#arima prediction interval first 
gPred <- ggplot() + geom_ribbon(data=arimaDF,aes(x=dateTS, ymin = Lo.80, ymax = Hi.80), alpha=.2)
#add arima prediction
gPred <- gPred + geom_line(data=arimaDF,aes(x=dateTS,y=Point.Forecast,color='arima_forecast'),size=1,linetype='dashed')

#add train and test sample
gPred <- gPred + 
  geom_line(data=filter(train,year>=2012),aes(x=dateTS,y=carbon_dioxide_ppm,color='train'),size=1) + 
  geom_line(data=filter(test,year>=2012),aes(x=dateTS,y=carbon_dioxide_ppm,color='test'),size=1,alpha=.5) + 
  scale_colour_manual(name="data",values=c(arima_forecast="#F21A00",train="#46ACC8",test="#EBCC2A")) + 
  theme(legend.position="top") +
  xlab('') + ylab('concentrations [ppm]')

gPred
```

##Exponential Smoothing State (ETS)
```{r}
#arima model
m_ets = ets(train_ts)
f_ets<-forecast(m_ets, 24)
etsDF<-convert_ts_df(f_ets)
```

```{r}
hPred <- ggplot() + geom_ribbon(data=etsDF,aes(x=dateTS, ymin = Lo.80, ymax = Hi.80), alpha=.2)
hPred <- hPred + geom_line(data=etsDF,aes(x=dateTS,y=Point.Forecast,color='ets_forecast'),size=1,linetype='dashed')

#add train and test sample
hPred <- hPred + 
  geom_line(data=filter(train,year>=2012),aes(x=dateTS,y=carbon_dioxide_ppm,color='train'),size=1) + 
  geom_line(data=filter(test,year>=2012),aes(x=dateTS,y=carbon_dioxide_ppm,color='test'),size=1,alpha=.5) + 
  scale_colour_manual(name="data",values=c(ets_forecast="#F21A00",train="#46ACC8",test="#EBCC2A")) + 
  theme(legend.position="top") +
  xlab('') + ylab('concentrations [ppm]')

hPred
```

##Removing NA's

```{r}
train_ts_na_lin<-na.interpolation(train_ts, option = "linear")
train_ts_na_spline<-na.interpolation(train_ts, option = "spline")
train_ts_na_stine<-na.interpolation(train_ts, option = "stine")
m_aa_lin = auto.arima(train_ts_na_lin)
m_aa_spline = auto.arima(train_ts_na_spline)
m_aa_stine = auto.arima(train_ts_na_stine)
res<-data.frame(
  names=c('arima','arima_linear_correction','arima_spline_correction','arima_stine_correction','ets'),
  aic=c(m_aa$aic,m_aa_lin$aic,m_aa_spline$aic,m_aa_stine$aic,m_ets$aic))
print(res)
```

Testing the arima model with the lower `aic` score :

```{r}
f_arima_lin<-forecast(m_aa_lin, 24)
arima_lin_DF<-convert_ts_df(f_arima_lin)
lPred <- ggplot() + geom_ribbon(data=arima_lin_DF,aes(x=dateTS, ymin = Lo.80, ymax = Hi.80), alpha=.2)
lPred <- lPred + geom_line(data=arima_lin_DF,aes(x=dateTS,y=Point.Forecast,color='arima_lin_forecast'),size=1,linetype='dashed')

#add train and test sample
lPred <- lPred + 
  geom_line(data=filter(train,year>=2012),aes(x=dateTS,y=carbon_dioxide_ppm,color='train'),size=1) + 
  geom_line(data=filter(test,year>=2012),aes(x=dateTS,y=carbon_dioxide_ppm,color='test'),size=1,alpha=.5) + 
  scale_colour_manual(name="data",values=c(arima_lin_forecast="#F21A00",train="#46ACC8",test="#EBCC2A")) + 
  theme(legend.position="top") +
  xlab('') + ylab('concentrations [ppm]')

lPred

```

While a linear interpolation with `auto.arima` to fill in the gaps gives the lower `aic`, the comparison with the `test` data is not much better.

<hr>
<strong>History :</strong>

* _version 1 : initial commit_ 
* _version 2 : added boxplot, comparison arima plot using ggplot_ 
* _version 3 : added ETS and time-series w/o NA's forecast_

<hr>